In [1]:
import pandas as pd
import glob
from datetime import date, timedelta
import numpy as np
from datetime import datetime
from pathlib import Path
from collections import OrderedDict
import polars as pl
import fastexcel
import os

In [2]:
def input_data(folder_path, sheet_name=None):
    file_paths = glob.glob(f"{folder_path}/*.xlsx") + glob.glob(f"{folder_path}/*.csv")
    df_list = []

    for file in file_paths:
        if file.endswith('.xlsx'):
            df = pl.read_excel(file, sheet_name=sheet_name)
        elif file.endswith('.csv'):
            try:
                df = pl.read_csv(file, encoding="utf-8")
            except:
                df = pl.read_csv(file, encoding="ISO-8859-1", ignore_errors=True)
        
        df = df.with_columns([
            pl.col(col).cast(pl.String) 
            for col in df.columns
        ])
        
        df_list.append(df)
    
    merged_df = pl.concat(df_list, how='vertical')

    return merged_df

today_temp = datetime.today().date()
today = today_temp.strftime('%b_%d_%Y')

In [3]:
first_glob_1 = "C:/Users/huuchinh.nguyen"
first_glob_2 = "C:/Users/ADMIN"

if os.path.exists(first_glob_1):
    first_glob = first_glob_1
elif os.path.exists(first_glob_2):
    first_glob = first_glob_2
else:
    raise FileNotFoundError(f"Neither {first_glob_1} nor {first_glob_2} exists.")

folder_paths = {
    # "input_agent_activity":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/STORAGE_INPUT_AGENT_ACTIVITY/2025',
    "input_agent_activity":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_AGENT_ACTIVITY_FOR_REPORT',
    "input_iex":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/OUTPUT_AGENT_IEX_FOR_REPORT',
    "input_pull_out":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Save/EXP Leave Management.xlsx',
    "input_hc_master":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Master Database - 2025.xlsx',
    "input_ramco_code":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_RAMCO_CODE',
    "input_ramco_ot":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/INPUT_OT_RAMCO_CODE',
    "rta_output":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/STORAGE_OUTPUT_RTA',
    "rta_intervals_output":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata/OUTPUT_AGENT_ACTIVITY_INTERVALS',
    "ramco_output":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Rawdata',
    "hc_extend_by_month":f'{first_glob}/Concentrix Corporation/WFM-Expedia-HCM - Branding files/Headcount/HC Extend by Month'
}

In [4]:
AGENT_ACTIVITY_INPUT = input_data(folder_paths["input_agent_activity"])
AGENT_ACTIVITY_INPUT = AGENT_ACTIVITY_INPUT[['Agent Email', 'Supervisor Email','Agent State','Number of Active Contacts','Productive Aux Flag (Yes / No)','Start Time','End Time','Total Time in seconds','Agent Business Location']]
AGENT_ACTIVITY_INPUT = AGENT_ACTIVITY_INPUT.unique()
remove_agent_state = ["End of Shift-IDLE", "Log out-IDLE","ENDOFSHIFT-IDLE"]
AGENT_ACTIVITY_INPUT = AGENT_ACTIVITY_INPUT.filter((pl.col('End Time').is_not_null()) & (~pl.col('Agent State').is_in(remove_agent_state)))
AGENT_ACTIVITY_INPUT = AGENT_ACTIVITY_INPUT.with_columns(pl.col('Total Time in seconds').str.replace(',', '').cast(pl.Float64).alias('Total Time in seconds'))
AGENT_ACTIVITY_INPUT = AGENT_ACTIVITY_INPUT.filter((pl.col('Total Time in seconds') < 3600*10))
# AGENT_ACTIVITY_INPUT = AGENT_ACTIVITY_INPUT.filter(~pl.col("Agent State").str.contains("Offline-IDLE"))

IEX = input_data(folder_paths["input_iex"])
IEX = IEX.unique()
IEX = IEX.with_columns([
    pl.col(['Date']).str.to_date("%Y-%m-%d", strict=False),
    pl.col(['Datetime_Fluctuate_Start_Shift','Datetime_Fluctuate_End_Shift','Datetime_First_Start_Shift','Datetime_First_End_Shift']).str.to_datetime("%Y-%m-%d %H:%M:%S.%f", strict=False),
    pl.col(['Night_Shift','Target','Unplanned','Planned','Roster Presented','Roster Scheduled']).cast(pl.Float64)])
columns_to_sec = ['Time_Of_Day','Open Time', 'Extra Time', 'Break Time', 'Lunch Time', 'Training', 'NCNS','AL','Target']
IEX = IEX.with_columns([(pl.col(col).fill_null(0).cast(pl.Float64) * 3600).alias(col) for col in columns_to_sec])
Night_Shift_1 = IEX[['Date','Email Id','Night_Shift']].unique()
Night_Shift_2 = Night_Shift_1.with_columns((pl.col('Date') - pl.duration(days=1)).alias('Previous Date'))
Night_Shift = Night_Shift_2.join(Night_Shift_1,left_on = ['Previous Date','Email Id'], right_on = ['Date','Email Id'], how = 'left')
Night_Shift = Night_Shift.rename({'Night_Shift_right':'Previous_Night_Shift'})
print(Night_Shift)
Previous_Date = IEX[['Date','Email Id','Datetime_First_Start_Shift','Shift Tracking']].unique()

PULL_OUT_DATA = pl.read_excel(folder_paths["input_pull_out"], sheet_name='Pull Out')
PULL_OUT_DATA = PULL_OUT_DATA.unique()
PULL_OUT_DATA = PULL_OUT_DATA.with_columns(
    pl.col('Pull-out duration (mins)').cast(pl.Float64).alias('Duration')
)
PULL_OUT_DATA = PULL_OUT_DATA.with_columns(pl.col('Duration') / 60)
PULL_OUT = PULL_OUT_DATA.group_by(['Pull-out Date','Emp ID']).agg([pl.col('Duration').sum().cast(pl.Float64).alias('total_time_pull_out')])

HC_MASTER_DATABASE = input_data(folder_paths["hc_extend_by_month"])
HC_MASTER_DATABASE = HC_MASTER_DATABASE.rename({'Date Start Week':'Week_Monday'})
HC_MASTER_DATABASE = HC_MASTER_DATABASE.with_columns(pl.col('Date').str.to_date("%Y-%m-%d", strict=False))

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_27960\3254034840.py:14: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col(['Datetime_Fluctuate_Start_Shift','Datetime_Fluctuate_End_Shift','Datetime_First_Start_Shift','Datetime_First_End_Shift']).str.to_datetime("%Y-%m-%d %H:%M:%S.%f", strict=False),


shape: (8_238, 5)
┌────────────┬────────────────────────────────┬─────────────┬───────────────┬──────────────────────┐
│ Date       ┆ Email Id                       ┆ Night_Shift ┆ Previous Date ┆ Previous_Night_Shift │
│ ---        ┆ ---                            ┆ ---         ┆ ---           ┆ ---                  │
│ date       ┆ str                            ┆ f64         ┆ date          ┆ f64                  │
╞════════════╪════════════════════════════════╪═════════════╪═══════════════╪══════════════════════╡
│ 2025-05-23 ┆ caotan.nguyen@concentrix.com   ┆ 1.0         ┆ 2025-05-22    ┆ 1.0                  │
│ 2025-05-07 ┆ thituongvy.nguyen1@concentrix. ┆ 1.0         ┆ 2025-05-06    ┆ 1.0                  │
│            ┆ …                              ┆             ┆               ┆                      │
│ 2025-05-09 ┆ hatuuyen.nguyen@concentrix.com ┆ 0.0         ┆ 2025-05-08    ┆ 0.0                  │
│ 2025-06-11 ┆ laikimthu.nguyen1@concentrix.c ┆ 0.0         ┆ 2025-06-10 

In [5]:
def process_ramco(folder_path, HC_MASTER_DATABASE):
    import polars as pl
    import glob

    file_paths = glob.glob(f"{folder_path}/Attendance Normal Month*.xlsx") + glob.glob(f"{folder_path}/Attendance Normal Month*.csv")
    df_list = []

    for file in file_paths:
        if file.endswith('.xlsx'):
            df_raw = pl.read_excel(file, sheet_id=0)
        elif file.endswith('.csv'):
            try:
                df_raw = pl.read_csv(file, encoding="utf-8")
            except:
                df_raw = pl.read_csv(file, encoding="ISO-8859-1", ignore_errors=True)

        columns_to_keep = ['OU', 'employee_code', 'employee_name', 'Employee Type'] + [f'{i}' for i in range(1, 32)]
        available_columns = [col for col in columns_to_keep if col in df_raw.columns]
        if not available_columns:
            continue

        df = df_raw.select(available_columns)

        if df.height < 2:
            continue

        header_row = df.row(0)

        new_columns = []
        for idx, col in enumerate(df.columns):
            if col in [str(i) for i in range(1, 32)]:
                new_col_name = header_row[idx]
                new_col_name_str = str(new_col_name) if new_col_name is not None else col
                new_columns.append(new_col_name_str)
            else:
                new_columns.append(col)
        df = df[2:]
        df = df.rename({old: new for old, new in zip(df.columns, new_columns)})
        df = df.filter(pl.col("OU").str.contains("Vietnam"))
        df = df.melt(id_vars=["OU", "employee_code", "employee_name", "Employee Type"],variable_name="Date",value_name="ramco_marked")
        df = df.filter((pl.col('Date') != "") & pl.col('Date').is_not_null())
        df = df.with_columns(
            pl.col("Date").str.strptime(pl.Date, "%d-%b-%Y", strict=False)
        )
        HC = HC_MASTER_DATABASE.select(['Date','OracleID', 'Email Id','Status'])
        df = df.join(HC, left_on=["Date","employee_code"], right_on=["Date","OracleID"], how='left')
        df = df.filter(pl.col('Status')=="Active")
        df = df.select(pl.all().exclude(["OU", "Email Id", "Status"]))
        # df = df.with_columns(pl.col('employee_code').cast(pl.Int32).alias('employee_code'))
        df = df.unique()
        df_list.append(df)

    if df_list:
        final_df = pl.concat(df_list, how='vertical')
    else:
        final_df = pl.DataFrame()

    return final_df

RAMCO = process_ramco(folder_paths["input_ramco_code"], HC_MASTER_DATABASE)
RAMCO = RAMCO.rename({'employee_code':'EID'})
RAMCO

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_27960\514442199.py:40: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df = df.melt(id_vars=["OU", "employee_code", "employee_name", "Employee Type"],variable_name="Date",value_name="ramco_marked")


EID,employee_name,Employee Type,Date,ramco_marked
str,str,str,date,str
"""103110468""","""THANH DAT HUA""","""Fixed Term Hire""",2025-04-18,"""WO"""
"""103111956""","""MINH QUAN LE""","""Fixed Term Hire""",2025-04-21,"""SL"""
"""102502718""","""GIA HAN DINH""","""Fixed Term Hire""",2025-04-22,"""AB"""
"""103124232""","""PHUONG TIEN DANG""","""Fixed Term Hire""",2025-04-26,"""WO"""
"""103109091""","""NGOC SON NGUYEN""","""Fixed Term Hire""",2025-04-27,"""SL"""
…,…,…,…,…
"""103101217""","""TIEN DAT HA""","""Fixed Term Hire""",2025-05-05,"""PR"""
"""102502737""","""THANH THUY DANG""","""Fixed Term Hire""",2025-05-07,"""PR"""
"""103169793""","""THI PHUONG DUNG LE""","""Fixed Term Hire""",2025-05-20,"""PR"""


In [6]:
def process_ramco_ot(folder_path, HC_MASTER_DATABASE):
    import polars as pl
    import glob

    file_paths = glob.glob(f"{folder_path}/OT Normal Month*.xlsx") + glob.glob(f"{folder_path}/OT Normal Month*.csv")
    df_list = []

    for file in file_paths:
        if file.endswith('.xlsx'):
            df_raw = pl.read_excel(file, sheet_id=0)
        elif file.endswith('.csv'):
            try:
                df_raw = pl.read_csv(file, encoding="utf-8")
            except:
                df_raw = pl.read_csv(file, encoding="ISO-8859-1", ignore_errors=True)

        columns_to_keep = ['OU', 'employee_code', 'employee_name', 'Employee Type','OT Type'] + [f'day_{i}' for i in range(1, 32)]
        available_columns = [col for col in columns_to_keep if col in df_raw.columns]
        if not available_columns:
            continue

        df = df_raw.select(available_columns)

        if df.height < 2:
            continue

        header_row = df.row(0)

        new_columns = []
        for idx, col in enumerate(df.columns):
            if col.startswith('day_'):
                try:
                    day_number = int(col.split('_')[1])
                except (IndexError, ValueError):
                    day_number = None

                if day_number is not None and 1 <= day_number <= 31:
                    new_col_name = header_row[idx]
                    new_col_name_str = str(new_col_name) if new_col_name is not None else col
                    new_columns.append(new_col_name_str)
                else:
                    new_columns.append(col)
            else:
                new_columns.append(col)
        df = df[2:]
        df = df.rename({old: new for old, new in zip(df.columns, new_columns)})
        df = df.filter(pl.col("OU").str.contains("Vietnam"))
        df = df.melt(id_vars=["OU", "employee_code", "employee_name", "Employee Type", "OT Type"],variable_name="Date",value_name="Temp")
        df = df.filter((pl.col('Date') != "") & pl.col('Date').is_not_null())
        df = df.with_columns(
            pl.col("Date").str.strptime(pl.Date, "%d-%b-%Y", strict=False)
        )
        HC = HC_MASTER_DATABASE.select(['Date','OracleID', 'Email Id','Status'])
        df = df.join(HC, left_on=["Date","employee_code"], right_on=["Date","OracleID"], how='left')
        df = df.filter(pl.col('Status')=="Active")
        df = df.select(pl.all().exclude(["OU", "Email Id", "Status"]))
        # df = df.with_columns(pl.col('employee_code').cast(pl.Int32).alias('employee_code'))
        df = df.unique()
        df_list.append(df)

    if df_list:
        final_df = pl.concat(df_list, how='vertical')
    else:
        final_df = pl.DataFrame()

    return final_df


RAMCO_OT_rawdata = process_ramco_ot(folder_paths['input_ramco_code'], HC_MASTER_DATABASE)

def split_temp_column(df):
    df = df.with_columns([
        pl.col("Temp").str.extract(r"(\d+\.\d+)", 1).alias("Hours"),
        pl.col("Temp").str.extract(r"-(\w+)", 1).alias("Status")
    ])
    
    df = df.with_columns([
        pl.when(pl.col("Temp").is_in(["0", None])).then(None).otherwise(pl.col("Hours")).alias("Hours"),
        pl.when(pl.col("Temp").is_in(["0", None])).then(None).otherwise(pl.col("Status")).alias("Status")
    ])
    
    return df

RAMCO_OT_rawdata = split_temp_column(RAMCO_OT_rawdata)

RAMCO_OT = RAMCO_OT_rawdata.with_columns(pl.col('Hours').str.to_decimal().cast(pl.Int64).alias("Hours"))
RAMCO_OT = RAMCO_OT.with_columns(pl.when(pl.col("Hours") > 0).then(pl.lit("(") + pl.col("Hours").cast(pl.Decimal(10, 1)) + pl.lit(" hrs") + pl.lit(" - ") + pl.col("Status") + pl.lit(")")).otherwise(pl.lit("")).alias("OT Status"))
RAMCO_OT = RAMCO_OT.group_by(["employee_code", "Date"]).agg([
    pl.col("OT Type").first().alias("ot_type"),
    pl.when(pl.col("Hours").is_not_null()).then(pl.col("Hours")).sum().alias("hours"),
    pl.when(pl.col("OT Type").str.contains("NSA")).then(pl.col("Status")).first().alias("nsa_authorize"),
    pl.when(pl.col("OT Type").str.contains("OT")).then(pl.col("Status")).first().alias("ot_authorize"),
    pl.col("OT Status").first().alias("ot_status")
]).fill_null("")
RAMCO_OT = RAMCO_OT.rename({'employee_code':'EID'})
RAMCO_OT = RAMCO_OT.with_columns(pl.col('EID').cast(pl.Utf8))

RAMCO_OT_rawdata

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_27960\3655215165.py:48: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df = df.melt(id_vars=["OU", "employee_code", "employee_name", "Employee Type", "OT Type"],variable_name="Date",value_name="Temp")


employee_code,employee_name,Employee Type,OT Type,Date,Temp,Hours,Status
str,str,str,str,date,str,str,str
"""103145144""","""QUANG DUY NGUYEN""","""Fixed Term Hire""","""OT3.0X""",2025-04-07,"""0""",null,null
"""103171084""","""NHAT HAO TRAN""","""Fixed Term Hire""","""NSA""",2025-04-05,"""0""",null,null
"""103131062""","""THI THANH PHUONG NGUYEN""","""Fixed Term Hire""","""OT3.0X""",2025-04-19,"""0""",null,null
"""102502737""","""THANH THUY DANG""","""Fixed Term Hire""","""NSA""",2025-04-18,"""0""",null,null
"""102830326""","""QUOC KHOI TRAN""","""Fixed Term Hire""","""OT3.0X""",2025-04-27,"""0""",null,null
…,…,…,…,…,…,…,…
"""102502746""","""HUU THO NGUYEN""","""Fixed Term Hire""","""OT3.0X""",2025-05-10,"""0""",null,null
"""102850964""","""NGUYEN NHU QUYNH TRAN""","""Fixed Term Hire""","""OT3.0X""",2025-05-21,"""0""",null,null
"""102478278""","""THANH TIEN VO""","""Fixed Term Hire""","""NSA""",2025-05-21,"""0""",null,null


In [7]:
AGENT_ACTIVITY_INPUT_FOR_NS = AGENT_ACTIVITY_INPUT.with_columns(pl.col('Start Time').cast(pl.Utf8).str.strptime(pl.Datetime).alias('Start Time'))
AGENT_ACTIVITY_INPUT_FOR_NS = AGENT_ACTIVITY_INPUT_FOR_NS.with_columns(pl.col('End Time').cast(pl.Utf8).str.strptime(pl.Datetime).alias('End Time'))
AGENT_ACTIVITY_INPUT_FOR_NS = AGENT_ACTIVITY_INPUT_FOR_NS.with_columns(pl.col('Start Time').dt.date().alias('Start Date'))
AGENT_ACTIVITY_INPUT_FOR_NS = AGENT_ACTIVITY_INPUT_FOR_NS.with_columns(pl.col('End Time').dt.date().alias('End Date'))
AGENT_ACTIVITY_INPUT_FOR_NS = AGENT_ACTIVITY_INPUT_FOR_NS.with_columns((pl.col('Start Date') + pl.duration(days=1)).alias('Next Date'))
AGENT_ACTIVITY_INPUT_FOR_NS = AGENT_ACTIVITY_INPUT_FOR_NS.with_columns((pl.col('Start Date') - pl.duration(days=1)).alias('Previous Date'))

ACTIVITY_MERGED_NIGHT_SHIFT = AGENT_ACTIVITY_INPUT_FOR_NS.join(Night_Shift[['Date','Email Id','Night_Shift','Previous_Night_Shift']],
                                                                 left_on=['Start Date','Agent Email'],
                                                                 right_on=['Date','Email Id'],
                                                                 how='left')
ACTIVITY_MERGED_NIGHT_SHIFT = ACTIVITY_MERGED_NIGHT_SHIFT.rename({'Start Date':'Date'})

def update_night_shift(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        pl.when(
            (pl.col('Night_Shift') == 0) & 
            (pl.col('Start Time').dt.time() < pl.time(17, 0)) & 
            (pl.col('Start Time').dt.time() >= pl.time(0, 0)) & 
            (pl.col('Previous_Night_Shift') == 1)
        ).then(False).otherwise(True).alias('Night_Shift_2_Check')
    )

    df = df.with_columns(
        pl.when(pl.col('Night_Shift_2_Check') == False)
        .then(1)
        .otherwise(pl.col('Night_Shift'))
        .alias('Night_Shift')
    )

    df = df.with_columns(
        pl.when((pl.col('Start Time').dt.hour() >= 0) & (pl.col('Start Time').dt.hour() < 12) & (pl.col('Previous_Night_Shift') == 1))
        .then(pl.col('Date') - pl.duration(days=1))
        
        .when((pl.col('Start Time').dt.hour() >= 0) & (pl.col('Start Time').dt.hour() < 12) & (pl.col('Night_Shift') == 1))
        .then(pl.col('Date') - pl.duration(days=1))
        
        .when((pl.col('Start Time').dt.hour() >= 0) & (pl.col('Start Time').dt.hour() < 18) & (pl.col('Night_Shift') == 0))
        .then(pl.col('Date'))
        
        .when((pl.col('Start Time').dt.hour() >= 18) & (pl.col('Night_Shift') == 1))
        .then(pl.col('Date'))
        
        .otherwise(pl.col('Date'))
        .alias('Date_Converted')
    )

    return df

ACTIVITY_MERGED_NIGHT_SHIFT = update_night_shift(ACTIVITY_MERGED_NIGHT_SHIFT)

Drop_NS = ACTIVITY_MERGED_NIGHT_SHIFT.drop('Night_Shift')
Merged_Drop_NS_With_IEX = Drop_NS.join(IEX[['Date','Email Id','First Shift']], left_on=['Date','Agent Email'], right_on=['Date','Email Id'], how='left')

Agent_Activity_Include_IEX = Merged_Drop_NS_With_IEX.filter(~pl.col('First Shift').is_null())

Agent_Not_IEX = Merged_Drop_NS_With_IEX.filter(pl.col('First Shift').is_null())
print(Agent_Not_IEX)
Agent_Not_IEX = Agent_Not_IEX.rename({'Date':'Start Date'})
Agent_Not_IEX = Agent_Not_IEX[['Agent Email', 'Supervisor Email','Agent State','Number of Active Contacts','Productive Aux Flag (Yes / No)','Start Time','End Time','Total Time in seconds','Agent Business Location', 'Start Date', 'End Date', 'Next Date', 'Previous Date']]
def add_tracker_time(Agent_Not_IEX: pl.DataFrame, agent_col: str, date_col: str, time_col: str, new_col: str, agg_func: str, time_threshold: int = 13, time_comparison: str = '>') -> pl.DataFrame:
    if time_comparison not in ['>', '<']:
        raise ValueError("Invalid comparison operator. Please use '>' or '<'.")
    if time_comparison == '>':
        filtered_data = Agent_Not_IEX.filter(pl.col(time_col).dt.hour() > time_threshold)
    else:
        filtered_data = Agent_Not_IEX.filter(pl.col(time_col).dt.hour() < time_threshold)
    if agg_func not in ['min', 'max']:
        raise ValueError("Invalid aggregation function. Please use 'min' or 'max'.")
    aggregation = getattr(pl.col(time_col), agg_func)().alias(new_col)
    min_max_time_per_group = filtered_data.group_by([agent_col, date_col]).agg(aggregation)
    Agent_Not_IEX = Agent_Not_IEX.join(min_max_time_per_group, on=[agent_col, date_col], how='left')
    return Agent_Not_IEX

AGENT_ACTIVITY_INPUT_1 = add_tracker_time(Agent_Not_IEX, agent_col="Agent Email", date_col="Start Date", time_col="Start Time", new_col="Min Date < 1pm", agg_func="min", time_comparison='<', time_threshold=13)
AGENT_ACTIVITY_INPUT_1 = add_tracker_time(AGENT_ACTIVITY_INPUT_1, agent_col="Agent Email", date_col="Start Date", time_col="Start Time", new_col="Max Date < 1pm", agg_func="max", time_comparison='<', time_threshold=13)
AGENT_ACTIVITY_INPUT_1 = add_tracker_time(AGENT_ACTIVITY_INPUT_1, agent_col="Agent Email", date_col="Start Date", time_col="Start Time", new_col="Min Date > 1pm", agg_func="min", time_comparison='>', time_threshold=13)
AGENT_ACTIVITY_INPUT_1 = add_tracker_time(AGENT_ACTIVITY_INPUT_1, agent_col="Agent Email", date_col="Start Date", time_col="Start Time", new_col="Max Date > 1pm", agg_func="max", time_comparison='>', time_threshold=13)
AGENT_ACTIVITY_INPUT_1 = AGENT_ACTIVITY_INPUT_1[['Agent Email','Start Date','Next Date','Min Date < 1pm','Max Date < 1pm','Min Date > 1pm','Max Date > 1pm']].unique()

AGENT_ACTIVITY_INPUT_2 = add_tracker_time(Agent_Not_IEX, agent_col="Agent Email", date_col="Start Date", time_col="Start Time", new_col="Min Date+1 < 4am", agg_func="min", time_comparison='<', time_threshold=4)
AGENT_ACTIVITY_INPUT_2 = add_tracker_time(AGENT_ACTIVITY_INPUT_2, agent_col="Agent Email", date_col="Start Date", time_col="Start Time", new_col="Max Date+1 < 1pm", agg_func="max", time_comparison='<', time_threshold=13)
AGENT_ACTIVITY_INPUT_2 = AGENT_ACTIVITY_INPUT_2[['Agent Email', 'Start Date','Min Date+1 < 4am','Max Date+1 < 1pm']].unique()

AGENT_ACTIVITY_INPUT_FINAL = AGENT_ACTIVITY_INPUT_1.join(AGENT_ACTIVITY_INPUT_2, left_on=['Agent Email', 'Start Date'],right_on=['Agent Email', 'Start Date'], how="left")

night_shift_condition = (
    ((pl.col('Min Date+1 < 4am').dt.hour() < 4) & (pl.col('Max Date+1 < 1pm').dt.hour() < 13))
)

AddedCustomNight_Shift_Check = AGENT_ACTIVITY_INPUT_FINAL.with_columns(
    pl.when(night_shift_condition).then(1).otherwise(0).alias('Night_Shift')
)
AddedCustomNight_Shift_Check = AddedCustomNight_Shift_Check[['Agent Email','Start Date','Next Date','Min Date < 1pm','Max Date < 1pm','Min Date > 1pm','Max Date > 1pm','Min Date+1 < 4am','Max Date+1 < 1pm','Night_Shift']]
AddedCustomNight_Shift_Check = AddedCustomNight_Shift_Check.unique()
AddedCustomNight_Shift_Check = AddedCustomNight_Shift_Check.rename({'Start Date':'Date'})
AddedCustomNight_Shift_Check = AddedCustomNight_Shift_Check.sort('Date')

Night_Shift_Tracker_1 = AddedCustomNight_Shift_Check[['Agent Email','Date','Night_Shift']]
Night_Shift_Tracker_2 = Night_Shift_Tracker_1.with_columns((pl.col('Date') - pl.duration(days=1)).alias('Previous Date'))
Night_Shift_Tracker = Night_Shift_Tracker_2.join(Night_Shift_Tracker_1[['Date','Agent Email','Night_Shift']],
                                                                 left_on=['Previous Date','Agent Email'],
                                                                 right_on=['Date','Agent Email'],
                                                                 how='left')
Night_Shift_Tracker = Night_Shift_Tracker.rename({'Night_Shift_right':'Previous_Night_Shift'})

ACTIVITY_MERGED_NIGHT_SHIFT_1 = Agent_Not_IEX.join(Night_Shift_Tracker[['Date','Agent Email','Night_Shift','Previous_Night_Shift']],
                                                                 left_on=['Start Date','Agent Email'],
                                                                 right_on=['Date','Agent Email'],
                                                                 how='left')
ACTIVITY_MERGED_NIGHT_SHIFT_1 = ACTIVITY_MERGED_NIGHT_SHIFT_1.rename({'Start Date':'Date'})


ACTIVITY_MERGED_NIGHT_SHIFT_1 = update_night_shift(ACTIVITY_MERGED_NIGHT_SHIFT_1)
Agent_Activity_Exclude_IEX = ACTIVITY_MERGED_NIGHT_SHIFT_1.drop('Night_Shift')

Agent_Activity_Include_IEX = Agent_Activity_Include_IEX.with_columns(
    pl.col('Previous_Night_Shift').cast(pl.Int32)
)

Agent_Activity_Exclude_IEX = Agent_Activity_Exclude_IEX.with_columns(
    pl.col('Previous_Night_Shift').cast(pl.Int32)
)

# AGENT_ACTIVITY_COMBINE = pl.concat([Agent_Activity_Include_IEX, Agent_Activity_Exclude_IEX])

shape: (1_194_094, 17)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────┐
│ Agent      ┆ Supervisor ┆ Agent      ┆ Number of ┆ … ┆ Previous_ ┆ Night_Shi ┆ Date_Conv ┆ First │
│ Email      ┆ Email      ┆ State      ┆ Active    ┆   ┆ Night_Shi ┆ ft_2_Chec ┆ erted     ┆ Shift │
│ ---        ┆ ---        ┆ ---        ┆ Contacts  ┆   ┆ ft        ┆ k         ┆ ---       ┆ ---   │
│ str        ┆ str        ┆ str        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ date      ┆ str   │
│            ┆            ┆            ┆ str       ┆   ┆ f64       ┆ bool      ┆           ┆       │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════╡
│ lehoaianh. ┆ vuphuonguy ┆ Available  ┆ 1         ┆ … ┆ null      ┆ true      ┆ 2025-04-1 ┆ null  │
│ vi@concent ┆ en.nguyen1 ┆ Chat       ┆           ┆   ┆           ┆           ┆ 6         ┆       │
│ rix.com    ┆ @concentri ┆            ┆           ┆   ┆           ┆

In [8]:
HC = HC_MASTER_DATABASE[['Year', 'Month', 'Week_Monday', 'Date','OracleID','IEX ID','Employee Name', 'Email Id','LOB','LOB_2','LOB_3',
                   'Alias', 'Supervisor Name', 'Wave', 'Detail Status', 'Status']].unique()
HC = HC.rename({'Date':'Date_Converted'})

COMPACT_IEX = IEX[['Date','Email Id','Target', 'Datetime_Fluctuate_Start_Shift', 'Datetime_Fluctuate_End_Shift', 
                   'Fluctuate Shift', 'Datetime_First_Start_Shift', 'Datetime_First_End_Shift', 
                   'First Shift','Time_Of_Day','Open Time', 'Extra Time', 'Break Time', 'Lunch Time', 'Training', 
                   'NCNS','AL','Unplanned','Planned','Roster Presented','Roster Scheduled', 'Night_Shift','Shift Tracking','OT Type','Combined OT Range', 'OT PreShift', 'OT PostShift']].unique()
COMPACT_IEX = COMPACT_IEX.rename({'Date':'Date_Converted'})

Agent_Activity_Include_IEX = Agent_Activity_Include_IEX.join(HC,left_on=['Date_Converted','Agent Email'],right_on=['Date_Converted','Email Id'],how='left')
Agent_Activity_Include_IEX = COMPACT_IEX.join(Agent_Activity_Include_IEX,left_on=['Date_Converted','Email Id'],right_on=['Date_Converted','Agent Email'],how='left')

Agent_Activity_Include_IEX_1 = Agent_Activity_Include_IEX.filter(pl.col("Week_Monday").is_not_null())
Agent_Activity_Include_IEX_2 = Agent_Activity_Include_IEX.filter((pl.col("First Shift").is_not_null()) & (pl.col("Week_Monday").is_null()))

standard_columns_1 = Agent_Activity_Include_IEX_1.columns[:Agent_Activity_Include_IEX_1.columns.index('Year')]
Agent_Activity_Include_IEX_2 = Agent_Activity_Include_IEX_2.select(standard_columns_1)
Agent_Activity_Include_IEX_2 = Agent_Activity_Include_IEX_2.join(HC,left_on=['Date_Converted','Email Id'],right_on=['Date_Converted','Email Id'],how='left')

Agent_Activity_Include_IEX = pl.concat([Agent_Activity_Include_IEX_1, Agent_Activity_Include_IEX_2]).unique()

Agent_Activity_Exclude_IEX = Agent_Activity_Exclude_IEX.join(HC,left_on=['Date_Converted','Agent Email'],right_on=['Date_Converted','Email Id'],how='left')
Agent_Activity_Exclude_IEX = Agent_Activity_Exclude_IEX.rename({'Agent Email': 'Email Id'})

columns_1 = set(Agent_Activity_Include_IEX.columns)
columns_2 = set(Agent_Activity_Exclude_IEX.columns)

missing_in_1 = columns_2 - columns_1
missing_in_2 = columns_1 - columns_2

for col in missing_in_1:
    Agent_Activity_Include_IEX = Agent_Activity_Include_IEX.with_columns(pl.lit(None).alias(col))

for col in missing_in_2:
    Agent_Activity_Exclude_IEX = Agent_Activity_Exclude_IEX.with_columns(pl.lit(None).alias(col))

standard_columns_2 = Agent_Activity_Include_IEX.columns

Agent_Activity_Exclude_IEX = Agent_Activity_Exclude_IEX.select(standard_columns_2)

ACTIVITY_DATE_CONVERTED_COMBINED = pl.concat([Agent_Activity_Include_IEX, Agent_Activity_Exclude_IEX]).unique()
print(ACTIVITY_DATE_CONVERTED_COMBINED.columns)

ACTIVITY_DATE_CONVERTED_COMBINED = ACTIVITY_DATE_CONVERTED_COMBINED.with_columns([
    pl.col("Year").cast(pl.Utf8),
    pl.col("OracleID").cast(pl.Utf8),
    pl.col("IEX ID").cast(pl.Utf8),
])

ACTIVITY_DATE_CONVERTED = ACTIVITY_DATE_CONVERTED_COMBINED.sort('Start Time')

['Date_Converted', 'Email Id', 'Target', 'Datetime_Fluctuate_Start_Shift', 'Datetime_Fluctuate_End_Shift', 'Fluctuate Shift', 'Datetime_First_Start_Shift', 'Datetime_First_End_Shift', 'First Shift', 'Time_Of_Day', 'Open Time', 'Extra Time', 'Break Time', 'Lunch Time', 'Training', 'NCNS', 'AL', 'Unplanned', 'Planned', 'Roster Presented', 'Roster Scheduled', 'Night_Shift', 'Shift Tracking', 'OT Type', 'Combined OT Range', 'OT PreShift', 'OT PostShift', 'Supervisor Email', 'Agent State', 'Number of Active Contacts', 'Productive Aux Flag (Yes / No)', 'Start Time', 'End Time', 'Total Time in seconds', 'Agent Business Location', 'Date', 'End Date', 'Next Date', 'Previous Date', 'Previous_Night_Shift', 'Night_Shift_2_Check', 'First Shift_right', 'Year', 'Month', 'Week_Monday', 'OracleID', 'IEX ID', 'Employee Name', 'LOB', 'LOB_2', 'LOB_3', 'Alias', 'Supervisor Name', 'Wave', 'Detail Status', 'Status']


In [9]:
ACTIVITY_DATE_CONVERTED['Productive Aux Flag (Yes / No)']

Productive Aux Flag (Yes / No)
str
null
null
null
null
null
…
"""Yes"""
"""Yes"""
"""Yes"""


In [10]:
GROUP_AND_SUM = ACTIVITY_DATE_CONVERTED.group_by([
    'Year','Month','Week_Monday','Date_Converted','Employee Name','Email Id','OracleID','IEX ID','Target',
    'Datetime_Fluctuate_Start_Shift','Datetime_Fluctuate_End_Shift','Fluctuate Shift',
    'Datetime_First_Start_Shift', 'Datetime_First_End_Shift', 'First Shift', 'Alias', 'LOB', 'LOB_2','LOB_3',
    'Supervisor Name','Wave','Detail Status','Status','Time_Of_Day','Open Time','Extra Time','Break Time',
    'Lunch Time','Training', 'NCNS','AL','Unplanned','Planned','Roster Presented','Roster Scheduled',
    'Night_Shift','Shift Tracking','OT Type','Combined OT Range','OT PreShift','OT PostShift'
]).agg([
    pl.col("Start Time").min().alias("start"),
    pl.col("End Time").max().alias("end"),
    pl.col("Total Time in seconds").sum().cast(pl.Float64).alias("duration"),

    pl.when(pl.col("Number of Active Contacts").is_not_null()).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("total_time_chat_handle"),
    pl.when(pl.col("Productive Aux Flag (Yes / No)") == "Yes").then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("sum_productive"),

    # BREAK (idle, Productive Aux Flag = No)
    pl.when(
        (pl.col("Agent State").is_in(["Break-IDLE", "Break"])) &
        (pl.col("Productive Aux Flag (Yes / No)") == "No")
    ).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("break"),

    # LUNCH (idle, Productive Aux Flag = No)
    pl.when(
        (pl.col("Agent State").is_in(["Lunch-IDLE", "Lunch"])) &
        (pl.col("Productive Aux Flag (Yes / No)") == "No")
    ).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("lunch"),

    # COACHING (idle, Productive Aux Flag = No)
    pl.when(
        (pl.col("Agent State").is_in(["Coaching-IDLE", "Coaching"])) &
        (pl.col("Productive Aux Flag (Yes / No)") == "No")
    ).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("coaching-idle"),

    # TRAINING (idle, Productive Aux Flag = No)
    pl.when(
        (pl.col("Agent State").is_in(["Training-IDLE", "Training"])) &
        (pl.col("Productive Aux Flag (Yes / No)") == "No")
    ).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("training-idle"),

    # OUTBOUND (idle, Productive Aux Flag = No)
    pl.when(
        (pl.col("Agent State").is_in(["Outbound Call-IDLE"])) &
        (pl.col("Productive Aux Flag (Yes / No)") == "No")
    ).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("outbound-idle"),

    # BREAK COUNT (idle, Productive Aux Flag = No)
    pl.when(
        (pl.col("Agent State").is_in(["Break-IDLE"])) & 
        (pl.col("Total Time in seconds") > 60) &
        (pl.col("Productive Aux Flag (Yes / No)") == "No")
    ).then(pl.col("Total Time in seconds")).count().cast(pl.Float64).alias("break_count"),

    # OTHER STATUS (idle, Productive Aux Flag = No, loại trừ các trạng thái trên)
    pl.when(
        (pl.col("Productive Aux Flag (Yes / No)") == "No") &
        (~pl.col("Agent State").is_in([
            "Break-IDLE", "Break", "Lunch-IDLE", "Lunch",
            "Coaching-IDLE", "Coaching", "Training-IDLE", "Training",
            "Outbound Call-IDLE"
        ]))
    ).then(pl.col("Total Time in seconds")).sum().cast(pl.Float64).alias("other_status"),
])


Added_Over_Break = GROUP_AND_SUM.with_columns(
    pl.when(pl.col("break").is_null()).then(0)
    .when(pl.col("Detail Status").str.contains("Production"))
        .then(pl.when((pl.col("break") - 30 * 60) < 0).then(0).otherwise(pl.col("break") - 30 * 60))
    .when(pl.col("Detail Status").str.contains("Nesting"))
        .then(pl.when((pl.col("break") - 15 * 60) < 0).then(0).otherwise(pl.col("break") - 15 * 60))
    .otherwise(0)
    .alias("over_break")
)

Added_Over_Lunch = Added_Over_Break.with_columns(
    pl.when(pl.col("lunch").is_null()).then(0)
    .when((pl.col("lunch") - 60 * 60) < 0).then(0)
    .otherwise(pl.col("lunch") - 60 * 60)
    .alias("over_lunch")
)

Added_Exceed_Break = Added_Over_Lunch.with_columns(
    pl.when(pl.col("break_count") == 0)
        .then(0)
    .when(pl.col("Detail Status").str.contains("Production"))
        .then(pl.when(pl.col("break_count")  <= 2).then(0).otherwise(pl.col("break_count") - 2))
    .when(pl.col("Detail Status").str.contains("Nesting"))
        .then(pl.when(pl.col("break_count") <= 1).then(0).otherwise(pl.col("break_count") - 1))
    .otherwise(0)
    .alias("exceed_break")
)

arrange_duration = Added_Exceed_Break.select([
    *[col for col in Added_Exceed_Break.columns if col != 'duration'],
    'duration'
])

added_HC_Actual = arrange_duration.with_columns(
    pl.when(pl.col("duration").is_null()).then(0)
    .otherwise(
        pl.when(pl.col("Detail Status").str.contains("Nesting"))
        .then(
            pl.when(pl.col("duration") >= 3*3600).then(1)
            .when((pl.col("duration") >= 2*3600) & (pl.col("duration") <= 3*3600)).then(0.5)
            .otherwise(0)
        )
        .otherwise(
            pl.when(pl.col("duration") >= 7*3600).then(1)
            .when((pl.col("duration") >= 3*3600) & (pl.col("duration") <= 6*3600)).then(0.5)
            .otherwise(0)
        )
    ).alias("hc_actual")
)

added_HC_Schedule = added_HC_Actual.with_columns(pl.when(
        (pl.col("First Shift").str.contains("-")) & (pl.col("Time_Of_Day") > 0) & (pl.col("Time_Of_Day") < 5*3600)).then(0.5).when(
        (pl.col("First Shift").str.contains("-")) & (pl.col("Time_Of_Day") >= 5*3600)).then(1)
    .when(pl.col("First Shift") == "AL").then(1).otherwise(0).cast(pl.Float64).alias("hc_schedule"))

added_Time_Late = added_HC_Schedule.with_columns(pl.when((pl.col("start") - pl.col("Datetime_Fluctuate_Start_Shift")).dt.total_seconds() > 180).then((pl.col("start") - pl.col("Datetime_Fluctuate_Start_Shift")).dt.total_seconds()).otherwise(0).cast(pl.Float64).alias("time_late"))
added_Time_Leave = added_Time_Late.with_columns(pl.when((pl.col("Datetime_Fluctuate_End_Shift") - pl.col("end")).dt.total_seconds() > 0).then((pl.col("Datetime_Fluctuate_End_Shift") - pl.col("end")).dt.total_seconds()).otherwise(0).cast(pl.Float64).alias("time_leave"))
addded_Lateness = added_Time_Leave.with_columns(pl.when(pl.col("start").is_not_null()).then(pl.when((pl.col("time_late").is_not_null()) & (pl.col("time_late") != 0)).then(1).otherwise(0)).otherwise(None).cast(pl.Float64).alias("lateness"))
addAdherenceTime = addded_Lateness.with_columns((pl.col("Target")- pl.col("time_late")-pl.col("over_break")-pl.col("over_lunch")).alias("adherence_time"))
merged_pull_out = addAdherenceTime.join(PULL_OUT[['Pull-out Date','Emp ID', 'total_time_pull_out']], left_on=['Date_Converted','OracleID'],right_on=['Pull-out Date','Emp ID'], how='left')

sec_to_hr = [
    'Target', 'Time_Of_Day', 'Open Time', 'Extra Time', 'Break Time', 'Lunch Time', 
    'Training', 'NCNS', 'AL', 'total_time_chat_handle', 'sum_productive', 'break', 
    'lunch', 'outbound-idle','coaching-idle','training-idle', 'over_break', 'over_lunch', 'duration', 'time_late', 
    'time_leave', 'adherence_time', 'total_time_pull_out', 'other_status'
]

converted_time = merged_pull_out.with_columns([(pl.col(col).fill_null(0).cast(pl.Float64) / 3600).alias(col) for col in sec_to_hr])
merged_ramco = converted_time.join(RAMCO[['EID','Date','ramco_marked']], left_on=['Date_Converted','OracleID'],right_on=['Date','EID'], how='left')
merged_ramco_ot = merged_ramco.join(RAMCO_OT[['EID','Date',"nsa_authorize","ot_authorize","hours","ot_status","ot_type"]], left_on=['Date_Converted','OracleID'],right_on=['Date','EID'], how='left')

from datetime import datetime
today = datetime.now().date()
added_today = merged_ramco_ot.with_columns(pl.lit(today).cast(pl.Date).alias("Today"))

added_days_count = added_today.with_columns([
    ((pl.col("Today") - pl.col("Date_Converted")).cast(pl.Float64) / (24 * 60 * 60 * 1000)).alias("days_count")
])

nm_group = added_days_count.with_columns([
    pl.when((pl.col("ramco_marked") == "NM"))
    .then(
        pl.when(pl.col("days_count").is_between(0, 2)).then(pl.lit("00-02 Days"))
        .when(pl.col("days_count").is_between(3, 5)).then(pl.lit("03-05 Days"))
        .when(pl.col("days_count").is_between(6, 10)).then(pl.lit("06-10 Days"))
        .when(pl.col("days_count").is_between(11, 30)).then(pl.lit(">10 Days"))
        .when(pl.col("days_count")> 30).then(pl.lit(">30 Days Unapproved"))
        .otherwise(None)
    )
    .otherwise(None).alias("nm_group")
])

nsa_group = nm_group.with_columns([
    pl.when((pl.col("ot_type").str.contains("NSA")) & (pl.col("nsa_authorize") == "Pending"))
    .then(
        pl.when(pl.col("days_count").is_between(0, 2)).then(pl.lit("00-02 Days"))
        .when(pl.col("days_count").is_between(3, 5)).then(pl.lit("03-05 Days"))
        .when(pl.col("days_count").is_between(6, 10)).then(pl.lit("06-10 Days"))
        .when(pl.col("days_count").is_between(11, 30)).then(pl.lit(">10 Days"))
        .when(pl.col("days_count")> 30).then(pl.lit(">30 Days Unapproved"))
        .otherwise(None)
    )
    .otherwise(None).alias("nsa_group")
])

ot_group = nsa_group.with_columns([
    pl.when((pl.col("ot_type").str.contains("OT")) & (pl.col("ot_authorize") == "Pending"))
    .then(
        pl.when(pl.col("days_count").is_between(0, 2)).then(pl.lit("00-02 Days"))
        .when(pl.col("days_count").is_between(3, 5)).then(pl.lit("03-05 Days"))
        .when(pl.col("days_count").is_between(6, 10)).then(pl.lit("06-10 Days"))
        .when(pl.col("days_count").is_between(11, 30)).then(pl.lit(">10 Days"))
        .when(pl.col("days_count")> 30).then(pl.lit(">30 Days Unapproved"))
        .otherwise(None)
    )
    .otherwise(None).alias("ot_group")
])

RTA_REPORT = ot_group
RTA_REPORT = RTA_REPORT.sort(['Date_Converted', 'start'], descending=[False, True])
RTA_REPORT = RTA_REPORT.unique()

print(RTA_REPORT.filter(
    (pl.col('Date_Converted').cast(pl.Date) == pl.lit('2025-05-12').str.strptime(pl.Date, "%Y-%m-%d")) & 
    (pl.col('Shift Tracking') == 'Off')
).select(['Date_Converted', 'Email Id', 'Shift Tracking']))


shape: (65, 3)
┌────────────────┬─────────────────────────────────┬────────────────┐
│ Date_Converted ┆ Email Id                        ┆ Shift Tracking │
│ ---            ┆ ---                             ┆ ---            │
│ date           ┆ str                             ┆ str            │
╞════════════════╪═════════════════════════════════╪════════════════╡
│ 2025-05-12     ┆ quockhang.lam@concentrix.com    ┆ Off            │
│ 2025-05-12     ┆ thaitrung.tang@concentrix.com   ┆ Off            │
│ 2025-05-12     ┆ huuhoang.nguyen@concentrix.com  ┆ Off            │
│ 2025-05-12     ┆ phanbaotuan.vo@concentrix.com   ┆ Off            │
│ 2025-05-12     ┆ daothanhtam.pham2@concentrix.c… ┆ Off            │
│ …              ┆ …                               ┆ …              │
│ 2025-05-12     ┆ thituongvy.tran@concentrix.com  ┆ Off            │
│ 2025-05-12     ┆ anny.doan@concentrix.com        ┆ Off            │
│ 2025-05-12     ┆ maihanhnhi.tran@concentrix.com  ┆ Off            │
│ 202

In [11]:
AGENT_ACTIVITY_SPLIT = ACTIVITY_DATE_CONVERTED.with_columns(pl.col('Start Time').dt.date().alias('Start Date'))
AGENT_ACTIVITY_SPLIT = AGENT_ACTIVITY_SPLIT.with_columns(pl.col('End Time').dt.date().alias('End Date'))
split_table = AGENT_ACTIVITY_SPLIT[['Year','Month','Week_Monday','Date_Converted','Date','Employee Name','Email Id','OracleID','IEX ID','LOB','Wave','Datetime_Fluctuate_Start_Shift','Datetime_Fluctuate_End_Shift','Fluctuate Shift','Start Date','End Date','Agent State','Start Time','End Time']]
split_table = split_table.rename({'Start Time': 'Datetime_Start_Action', 'End Time': 'Datetime_End_Action'})
split_table = split_table.with_columns([((pl.col(f'Datetime_{col}_Action').dt.hour() + pl.col(f'Datetime_{col}_Action').dt.minute() / 60 + pl.col(f'Datetime_{col}_Action').dt.second() / 3600).round(2)).alias(f'{col}_Time_Hours') for col in ['Start', 'End']])
split_table = split_table.with_columns(pl.when(pl.col('End_Time_Hours') < pl.col('Start_Time_Hours')).then(pl.col('End_Time_Hours') + 24).otherwise(pl.col('End_Time_Hours')).alias('End_Time_Hours'))
for col in ['Start', 'End']:
    split_table = split_table.with_columns(
        pl.when(pl.col(f"{col}_Time_Hours") * 2 % 1 == 0)
          .then(pl.col(f"{col}_Time_Hours").round(1))
          .otherwise((pl.col(f"{col}_Time_Hours") * 2).floor() / 2)
          .alias(f"{col}_Time_Half_Rounding")
    )
split_table = split_table.with_columns(
    pl.when(
        pl.col('End_Time_Half_Rounding').is_not_nan() & 
        pl.col('Start_Time_Half_Rounding').is_not_nan()
).then((pl.col('End_Time_Half_Rounding') - pl.col('Start_Time_Half_Rounding')) * 2).otherwise(None).cast(pl.Int64).alias('Number_Split'))
split_table = split_table.with_columns(pl.col('Start_Time_Half_Rounding').alias('Time_1'))
split_table = split_table.with_columns((pl.col('Time_1') + 0.5).alias('Time_2') )
split_table = split_table.with_columns(
    pl.col('Number_Split').cast(pl.Int32).repeat_by(pl.col('Number_Split')).alias('replicate')
)
split_table = split_table.explode('replicate')
split_table = split_table.with_columns(
    pl.concat_str([pl.col("Date_Converted").cast(pl.Utf8), pl.col("IEX ID").cast(pl.Utf8), pl.col("Datetime_Start_Action").cast(pl.Utf8)], separator=" - ").alias("Unique_Count")).with_columns(
        (pl.arange(0, pl.count()).over(["Date_Converted", "IEX ID", "Unique_Count"]) + 1).alias("Custom")).with_columns(pl.col('Custom')- 1)
split_table = split_table.with_columns((pl.col('Time_1') + pl.col('Custom') * 0.5).alias('Time_1_New'))
split_table = split_table.with_columns((pl.col('Time_2') + pl.col('Custom') * 0.5).alias('Time_2_New'))
split_table = split_table.with_columns((((pl.col('End_Time_Hours') * 2).floor() + 1) / 2).cast(pl.Float32).alias('Rounded_End_Time_Hours'))
split_table = split_table.with_columns(
    pl.when(pl.col("Time_1_New") > pl.col("Start_Time_Hours")).then(pl.col("Time_1_New")).otherwise(pl.col("Start_Time_Hours")).alias("Start_Time_Number"),
    pl.when(pl.col("Time_2_New") < pl.col("End_Time_Hours")).then(pl.col("Time_2_New")).otherwise(pl.col("End_Time_Hours")).alias("End_Time_Number")
)
split_table = split_table.with_columns(
    pl.when(pl.col('Start Date') == pl.col('End Date')).then(pl.col('Start Date')).otherwise(
        pl.when((pl.col('Start_Time_Number') >= 24) & (pl.col('Start Date') != pl.col('End Date'))).then(pl.col('Start Date') + pl.duration(days=1)).otherwise(pl.col('Start Date'))
    ).alias('New_Day_Start_Time')
)
split_table = split_table.with_columns(pl.when(pl.col('End_Time_Number') < pl.col('Start_Time_Number')).then(pl.col('End Date') - pl.duration(days=1)).otherwise(pl.col('End Date')).alias('New_Day_End_Time'))
for col in ['Start', 'End']:
    split_table = split_table.with_columns(
        ((pl.col(f'{col}_Time_Number') - 24 * (pl.col(f'{col}_Time_Number') >= 24).cast(pl.Int32)) % 24).alias(f'{col}_Time_Number')
    )
def number_to_time(number):
    if number is None:
        return None
    hours = int(np.floor(number))
    minutes = int(np.floor((number - hours) * 60))
    seconds = int(np.floor((number - hours - minutes / 60) * 3600))
    return f"{hours:02}:{minutes:02}:{seconds:02}"
def number_to_time(df, col):
    hours = (df[col] // 1).cast(pl.Int32)
    minutes = ((df[col] % 1) * 60).cast(pl.Int32)
    seconds = (((df[col] % 1) * 60) % 1 * 60).cast(pl.Int32)
    return (pl.concat_str([hours.cast(pl.Utf8).str.zfill(2),minutes.cast(pl.Utf8).str.zfill(2),seconds.cast(pl.Utf8).str.zfill(2)], separator=':'))
for col in ['Start', 'End']:
    split_table = split_table.with_columns(
        number_to_time(split_table, f'{col}_Time_Number').alias(f'{col}_Time')
    )
split_table = split_table.with_columns(pl.when(pl.col('Time_1_New') >= 24).then(pl.col('Time_1_New')-24).otherwise(pl.col('Time_1_New')).alias('Intervals'))
split_table = split_table.with_columns(number_to_time(split_table, 'Intervals'))

def hms_to_duration(df: pl.DataFrame, column: str) -> pl.DataFrame:
    return df.with_columns([
        pl.col(column).str.split(":").alias(f"{column}_parts")
    ]).with_columns([
        (pl.col(f"{column}_parts").list.get(0).cast(pl.Int64) * pl.duration(hours=1) +
         pl.col(f"{column}_parts").list.get(1).cast(pl.Int64) * pl.duration(minutes=1) +
         pl.col(f"{column}_parts").list.get(2).cast(pl.Int64) * pl.duration(seconds=1)
        ).alias(f"{column}_duration")
    ]).drop(f"{column}_parts")

split_table = hms_to_duration(split_table, "Intervals")

split_table = split_table.with_columns([
    (pl.concat_str([pl.col("New_Day_Start_Time"), pl.lit(" "), pl.col("Start_Time")]).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("Datetime_Start_Time")),
    (pl.concat_str([pl.col("New_Day_End_Time"), pl.lit(" "), pl.col("End_Time")]).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("Datetime_End_Time"))
])
split_table = split_table.filter(pl.col("Datetime_Start_Time") < pl.col("Datetime_End_Time"))
# split_table = split_table['Year','Month','Week Begin','Date_Converted','Start Date','Employee Name','OracleID','IEX ID','Wave','LOB','Agent State','Intervals','Datetime_Fluctuate_Start_Shift','Datetime_Fluctuate_End_Shift','Fluctuate Shift','Datetime_Start_Time','Datetime_End_Time']
split_table = split_table.group_by(['Year','Month','Week_Monday','Date_Converted','Start Date','OracleID','IEX ID','Employee Name','Email Id','LOB','Wave','Datetime_Fluctuate_Start_Shift','Datetime_Fluctuate_End_Shift','Fluctuate Shift','Intervals','Agent State']).agg(
    [
        pl.col('Datetime_Start_Time').min().cast(pl.Datetime),
        pl.col('Datetime_End_Time').max().cast(pl.Datetime) 
    ])
split_table = split_table.with_columns(
    ((pl.col("Datetime_End_Time") - pl.col("Datetime_Start_Time")).dt.total_seconds() / 3600).alias('Duration')
)
print(split_table)
RTA_EXTEND = split_table

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_11912\348970879.py:27: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.arange(0, pl.count()).over(["Date_Converted", "IEX ID", "Unique_Count"]) + 1).alias("Custom")).with_columns(pl.col('Custom')- 1)


shape: (665_975, 19)
┌──────┬────────┬─────────────┬─────────────┬───┬─────────────┬────────────┬────────────┬──────────┐
│ Year ┆ Month  ┆ Week_Monday ┆ Date_Conver ┆ … ┆ Agent State ┆ Datetime_S ┆ Datetime_E ┆ Duration │
│ ---  ┆ ---    ┆ ---         ┆ ted         ┆   ┆ ---         ┆ tart_Time  ┆ nd_Time    ┆ ---      │
│ str  ┆ str    ┆ str         ┆ ---         ┆   ┆ str         ┆ ---        ┆ ---        ┆ f64      │
│      ┆        ┆             ┆ date        ┆   ┆             ┆ datetime[μ ┆ datetime[μ ┆          │
│      ┆        ┆             ┆             ┆   ┆             ┆ s]         ┆ s]         ┆          │
╞══════╪════════╪═════════════╪═════════════╪═══╪═════════════╪════════════╪════════════╪══════════╡
│ 2025 ┆ May-25 ┆ 2025-05-12  ┆ 2025-05-15  ┆ … ┆ Available   ┆ 2025-05-15 ┆ 2025-05-15 ┆ 0.5      │
│      ┆        ┆             ┆             ┆   ┆ Chat        ┆ 07:00:00   ┆ 07:30:00   ┆          │
│ 2025 ┆ Apr-25 ┆ 2025-04-07  ┆ 2025-04-08  ┆ … ┆ Offline     ┆ 2025-0

In [12]:
# columns = RTA_REPORT.columns
# data_types = [str(RTA_REPORT[col].dtype) for col in columns]

# schema_df = pd.DataFrame({
#     'Tên Cột': columns,
#     'Kiểu Dữ Liệu': data_types
# })

# schema_df.to_excel('output.xlsx', index=False, engine='openpyxl')

In [13]:
for Week_Monday, group in RTA_REPORT.group_by('Week_Monday'):
    Week_Monday_str = Week_Monday[0]
    file_name = f'{Week_Monday_str}.xlsx'
    file_path = os.path.join(folder_paths["rta_output"], file_name)
    group.write_excel(file_path)

for Week_Monday, group in RTA_EXTEND.group_by('Week_Monday'):
    Week_Monday_str = Week_Monday[0]
    file_name = f'{Week_Monday_str}.xlsx'
    file_path = os.path.join(folder_paths["rta_intervals_output"], file_name)
    group.write_excel(file_path)